In [2]:
# coding: utf-8

In [3]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
import sklearn as sk
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import OneHotEncoder
import os
from keras.utils import np_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Reshape, Flatten
from keras.layers import Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import classification_report
from keras.preprocessing.text import hashing_trick
from keras.preprocessing.text import text_to_word_sequence
from keras.regularizers import l2

from keras.preprocessing.text import Tokenizer
from keras.layers import Conv2D, MaxPooling2D

from keras.models import load_model

Using TensorFlow backend.


In [4]:
filenames = ["reviews_Home_and_Kitchen_5.json"]
filename = "reviews_Home_and_Kitchen_5.json"
t = Tokenizer()
max_features = 20000
maxlen = 100
batch_size = 32

dictionary_size = 0
embed_dim = 300
num_hidden_unit_lstm = 300
num_filter = 100
window_size = 3
pool_size = 2
batch_size = 10
learning_rate = 0.1
#AdaDelta
dropout_embed = 0.5
dropout_blstm = 0.2
dropout_penultimate = 0.4
l2_lambda = 0.000001

randomShuffle = ShuffleSplit(n_splits=1, test_size=.2, random_state=0)

In [5]:
data = pd.read_json(filename, lines=True, orient="frame")

ValueError: Expected object or value

In [ ]:
data.head(10)

In [ ]:
data['overall'] = np.where(data['overall'] < 3, 0, data.overall)
data['overall'] = np.where(data['overall'] == 3, 1, data.overall)
data['overall'] = np.where(data['overall'] > 3, 2, data.overall)

In [ ]:
data.head(10)

In [ ]:
label = list(data.iloc[:, 2].values)
for j in range(3):
    print(str(j) + ": " + str(label.count(j)) + "\t\t" + str(round(label.count(j) / len(label) * 100, 2)) + "%")

plt.plot([0, 1, 2], [label.count(0), label.count(1), label.count(2)], 'ro')
plt.show()

In [ ]:
def getEmbeddingMatrix(vocabulary_size):
    embeddings_index = dict()
    f = open('glove.6b/glove.6B.300d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
    embedding_matrix = np.zeros((vocabulary_size, embed_dim))
    for word, i in t.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

### 2 classes(0: 1-2 star, 1: 4-5 star)

In [21]:
def getXY(filename, num_class):
    data = pd.read_json(filename, lines=True, orient="frame")
    data['overall'] = np.where(data['overall'] < 3, 0, data.overall)
    data['overall'] = np.where(data['overall'] > 3, 1, data.overall)
    
    # down sample
    data_0 = data.loc[data['overall'] == 0]
    data_1 = data.loc[data['overall'] == 1].sample(len(data_0))
    
    data = data_0.append(data_1).sample(frac=1)
    
    good_columns = [3]
    onehot_encoder = OneHotEncoder(sparse=False)
    data_X = data.iloc[:, good_columns]
    data_Y = data.iloc[:, 2]
    
    reviewTextList = data.reviewText.values
    t.fit_on_texts(reviewTextList)
    text = t.texts_to_sequences(reviewTextList)
    text = sequence.pad_sequences(text, maxlen=maxlen)
    print('text shape:', text.shape)
    vocabulary_size = len(t.word_index) + 1
    print('vocabulary size:', vocabulary_size)
    
    data_X = np.array(text)
    data_Y = np_utils.to_categorical(data_Y, num_class)
    return data_X, data_Y, vocabulary_size

In [6]:
def bidirectionalLSTMCNNModel(containsGlove, num_class, str_loss, x_train, x_test, y_train, y_test, batch_size, epochs, vocabulary_size, embedding_matrix, name_model):
    print("x_train: ", x_train.shape)
    print("y_train: ", y_train.shape)
    print("x_test: ", x_test.shape)
    print("y_test: ", y_test.shape)
#     print("x_train:",x_train[0])
#     print("x_test:",x_test[0])
#     print('y_train:',y_train[0])
#     print('y_test',y_test[0])
    model = Sequential()
    print("vocabulary_size:", vocabulary_size)
    if (containsGlove == False):
        model.add(Embedding(input_dim=vocabulary_size, output_dim=embed_dim, input_length=maxlen))
    else:
        model.add(Embedding(input_dim=vocabulary_size, output_dim=embed_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False))
    model.add(Dropout(0.5))
    model.add(
        Bidirectional(
            LSTM(
                num_hidden_unit_lstm, 
                return_sequences=True, 
                 kernel_regularizer=l2(l2_lambda)
            ), 
            merge_mode="sum"
        ))
    model.add(Dropout(0.2))
    model.add(Reshape((maxlen, embed_dim, 1)))
    model.add(Conv2D(num_filter,
                     kernel_size=(3,3),
                     padding='valid',
                     activation='relu',
                     strides=1,
                      kernel_regularizer=l2(l2_lambda)
                    ))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(num_class, kernel_regularizer=l2(l2_lambda)))
    model.add(Activation('sigmoid'))

    print(model.summary())
    model.compile(loss=str_loss, 
                  optimizer='Adadelta',
                  metrics=['accuracy'])
    
    print('Train...')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=[x_test, y_test])
    
    score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
    print('Test score:', score)
    print('Test accuracy:', acc)
    y_test = np.argmax(y_test, axis=1)
    y_pred = model.predict_classes(x_test)
    print(classification_report(y_test, y_pred))
    model.save(name_model)
    del model

### Compare without and with GloVe Embedding

In [12]:
vocabulary_size = 0
num_class = 2
for filename in filenames:
    print("-----------------------------")
    print(filename + ":")
    X, Y, vocabulary_size = getXY(filename, num_class)
    embedding_matrix = getEmbeddingMatrix(vocabulary_size)
    for train_index, test_index in randomShuffle.split(X):
        X_train = X[train_index]
        Y_train = Y[train_index, :]
        X_test = X[test_index]
        Y_test = Y[test_index, :]
        
        bidirectionalLSTMCNNModel(False, num_class, 'categorical_crossentropy', X_train, X_test, Y_train, Y_test, 10, 1, vocabulary_size, embedding_matrix, filename + 'BLSTM+CNN_without.h5')
        bidirectionalLSTMCNNModel(True, num_class, 'categorical_crossentropy', X_train, X_test, Y_train, Y_test, 10, 1, vocabulary_size, embedding_matrix, filename + 'BLSTM+CNN_with.h5')

-----------------------------
Home_and_Kitchen_5.json:
text shape: (102838, 100)
vocabulary size: 72649
Loaded 400000 word vectors.
x_train:  (82270, 100)
y_train:  (82270, 2)
x_test:  (20568, 100)
y_test:  (20568, 2)
vocabulary_size: 72649
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          21794700  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 300)          1442400   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 100, 300, 1)       0         
_________________________________

### Run without GloVe for 5 epochs

In [23]:
vocabulary_size = 0
num_class = 2
for filename in filenames:
    print("-----------------------------")
    print(filename + ":")
    X, Y, vocabulary_size = getXY(filename, num_class)
    for train_index, test_index in randomShuffle.split(X):
        X_train = X[train_index]
        Y_train = Y[train_index, :]
        X_test = X[test_index]
        Y_test = Y[test_index, :]
        
        bidirectionalLSTMCNNModel(False, num_class, 'categorical_crossentropy', X_train, X_test, Y_train, Y_test, 10, 5, vocabulary_size, embedding_matrix, 'BLSTM+CNN.h5')

-----------------------------
reviews_Home_and_Kitchen_5.json:
text shape: (102838, 100)
vocabulary size: 72038
Loaded 400000 word vectors.
x_train:  (82270, 100)
y_train:  (82270, 2)
x_test:  (20568, 100)
y_test:  (20568, 2)
vocabulary_size: 72038
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          21611400  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 300)          1442400   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 100, 300, 1)       0         
_________________________

ImportError: `save_model` requires h5py.

In [1]:
model = Sequential()
model.save('new.h5')

NameError: name 'Sequential' is not defined

In [ ]:
model = load_model('BLSTM+CNN.h5')

In [7]:
def getXY(filename, num_class):
    cat = [0, 1]
    def cat_y(y):
        if y<=2.0:
            return cat[0]
        elif y>=4.0:
            return cat[1]

    def get_reviews(path, n_samples):
        dt = {}
        i=0
        with open(path) as f:
            for d in f.readlines():
                dt[i] = eval(d)
                i += 1
        df = pd.DataFrame.from_dict(dt, orient='index')[['reviewText','overall']]
        print("before: ", len(df))
        df = df.drop(df[df.overall == 3.0].index)
        print("after: ", len(df))
        df = df[df['reviewText'].apply(lambda x: len(x.split())>=45)]
        df['bucket'] = df['overall'].apply(cat_y)

        df = df.groupby('bucket').apply(lambda x: x.sample(n=n_samples))
        return df

    data = get_reviews('data/reviews_Home_and_Kitchen_5.json', 10000)
    
    good_columns = [3]
    onehot_encoder = OneHotEncoder(sparse=False)
    data_X = data.iloc[:, 0]
    data_Y = data.iloc[:, 2]
    
    reviewTextList = data.reviewText.values
    t.fit_on_texts(reviewTextList)
    text = t.texts_to_sequences(reviewTextList)
    text = sequence.pad_sequences(text, maxlen=maxlen)
    print('text shape:', text.shape)
    vocabulary_size = len(t.word_index) + 1
    print('vocabulary size:', vocabulary_size)
    
    data_X = np.array(text)
    data_Y = np_utils.to_categorical(data_Y, num_class)
    return data_X, data_Y, vocabulary_size

In [10]:
vocabulary_size = 0
num_class = 2
for filename in filenames:
    print("-----------------------------")
    print(filename + ":")
    X, Y, vocabulary_size = getXY(filename, num_class)
    for train_index, test_index in randomShuffle.split(X):
        X_train = X[train_index]
        Y_train = Y[train_index, :]
        X_test = X[test_index]
        Y_test = Y[test_index, :]
        
        bidirectionalLSTMCNNModel(False, num_class, 'categorical_crossentropy', X_train, X_test, Y_train, Y_test, 10, 5, vocabulary_size, [], 'BLSTM+CNN.h5')

-----------------------------
reviews_Home_and_Kitchen_5.json:
before:  551682
after:  506623
text shape: (20000, 100)
vocabulary size: 59641
x_train:  (16000, 100)
y_train:  (16000, 2)
x_test:  (4000, 100)
y_test:  (4000, 2)
vocabulary_size: 59641
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          17892300  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 300)          1442400   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 100, 300, 1)       0         
_________________________